<a href="https://colab.research.google.com/github/AugustvonMackensen/AI_colab/blob/main/leNet_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LeNet-5 : Classification by Covolutions and Sampling repeatly

[1] Import the packages

In [1]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Layer
from keras import backend as K
import numpy as np

[2] Prepare the Data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# input image : 28 X 28, Channel : 1 because it is black and white
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

11490434/11490434 [==============================] - 0s 0us/step


[3] Data Padding and normalization

In [3]:
#np.pad() : add the paddings. Padding is used to adjust the size of the image
#               or  preserve the edge information.
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Normalize the data(the value must be float number between 0 and 1)
x_train = x_train / 255
x_test = x_test / 255

[4] Vectorize the data for the label

convert y_train and y_test to 'one-hot encoding'
to_categorical(): a function that converts integer data into one-hot encoding

In [4]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

[5] Create RBF Layer

In [5]:
class RBFLayer(Layer):
  def __init__(self, units, gamma, **kwargs):
    super(RBFLayer, self).__init__(**kwargs)
    self.units = units
    self.gamma = K.cast_to_floatx(gamma) # cast_to_floatx : Cast a Numpy array to the default Keras float type.

  def build(self, input_shape):
    self.mu = self.add_weight(name='mu', shape=(int(input_shape[1]), self.units),
                              initializer='uniform',
                              trainable=True)
    super(RBFLayer, self).build(input_shape)

  def call(self, inputs):
    diff = K.expand_dims(inputs) - self.mu
    l2 = K.sum(K.pow(diff,2), axis=1)
    res = K.exp(-1 * self.gamma * l2)
    return res

  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.units)

[6] Define the Model

In [6]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5,5), activation='tanh', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, kernel_size=(5,5), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(RBFLayer(10, 0.5))

[7] Fit

In [7]:
model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
1875/1875 [==============================] - 61s 31ms/step - loss: 0.0292 - accuracy: 0.8363 - val_loss: 0.0109 - val_accuracy: 0.9739
Epoch 2/20
1875/1875 [==============================] - 46s 24ms/step - loss: 0.0089 - accuracy: 0.9731 - val_loss: 0.0065 - val_accuracy: 0.9764
Epoch 3/20
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0055 - accuracy: 0.9774 - val_loss: 0.0049 - val_accuracy: 0.9799
Epoch 4/20
1875/1875 [==============================] - 46s 24ms/step - loss: 0.0042 - accuracy: 0.9820 - val_loss: 0.0038 - val_accuracy: 0.9827
Epoch 5/20
1875/1875 [==============================] - 46s 25ms/step - loss: 0.0035 - accuracy: 0.9843 - val_loss: 0.0038 - val_accuracy: 0.9838
Epoch 6/20
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0030 - accuracy: 0.9865 - val_loss: 0.0041 - val_accuracy: 0.9820
Epoch 7/20
1875/1875 [==============================] - 50s 26ms/step - loss: 0.0027 - accuracy: 0.9879 - val_loss: 0.0029 -

[8] Check the accuracy score

In [8]:
score = model.evaluate(x_test, y_test)
print('test accuracy rate : ', score[1])

313/313 [==============================] - 3s 9ms/step - loss: 0.0022 - accuracy: 0.9881
test accuracy rate :  0.988099992275238
